## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [ ]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [ ]:
### Imprima os três primeiros registros da tabela tracks

from sqlalchemy import select

res = session.scalars(select(Base.classes.tracks).limit(3))
for track in res:
    print(track.TrackId, track.Name)


In [ ]:

### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.

stmt = select(Base.classes.tracks.Name, Base.classes.albums.Title).join(Base.classes.albums).limit(20)
for row in session.execute(stmt):
    print(f"{row.Name} - {row.Title}")

In [ ]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.

stmt = select(Base.classes.invoice_items, Base.classes.tracks.Name)\
    .join(Base.classes.tracks)\
    .limit(10)

for row in session.execute(stmt):
    print(f"Track: {row.Name} | Quantity: {row.Quantity}")


In [ ]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.

from sqlalchemy import func

stmt = select(Base.classes.tracks.Name, func.sum(Base.classes.invoice_items.Quantity).label('total_sold'))\
    .join(Base.classes.invoice_items)\
    .group_by(Base.classes.tracks.TrackId)\
    .order_by(func.sum(Base.classes.invoice_items.Quantity).desc())\
    .limit(10)

for row in session.execute(stmt):
    print(f"{row.Name}: {row.total_sold}")

In [ ]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists

stmt = select(Base.classes.artists.Name, func.sum(Base.classes.invoice_items.Quantity).label('total_sold'))\
    .join(Base.classes.albums, Base.classes.artists.ArtistId == Base.classes.albums.ArtistId)\
    .join(Base.classes.tracks, Base.classes.albums.AlbumId == Base.classes.tracks.AlbumId)\
    .join(Base.classes.invoice_items, Base.classes.tracks.TrackId == Base.classes.invoice_items.TrackId)\
    .group_by(Base.classes.artists.ArtistId)\
    .order_by(func.sum(Base.classes.invoice_items.Quantity).desc())\
    .limit(10)

for row in session.execute(stmt):
    print(f"{row.Name}: {row.total_sold}")